# LangGraph Advanced – Part 5️⃣: Multi‑Step Support Workflow

Workflow đa bước (user → retrieve → answer → moderation → approve).

In [ ]:
%pip install -q langgraph langchain langchain-anthropic chromadb tiktoken

In [ ]:
from langgraph.graph import StateGraph, MessageState
from langchain_anthropic import ChatAnthropic
from langchain.schema import HumanMessage
import random

llm = ChatAnthropic(model_name="claude-3-haiku-20240307", temperature=0.3)

class S(MessageState):
    question:str|None=None
    answer:str|None=None

def ask(_:S):
    return S(question=input("Ask: "))

def retrieve(state:S):
    state.context="fake-doc"
    return state

def answer(state:S):
    state.answer = llm([HumanMessage(content=f"{state.question}\nCONTEXT:{state.context}")]).content
    return state

def moderate(state:S):
    if "bad" in state.answer.lower():
        state.answer="[moderated]"
    return state

g = StateGraph(S)
g.add_node("ask", ask)
g.add_node("ret", retrieve)
g.add_node("ans", answer)
g.add_node("mod", moderate)
g.set_entry_point("ask")
g.add_edge("ask","ret").add_edge("ret","ans").add_edge("ans","mod")
g.compile().invoke(S())
